# deploy

## h5 (\Web)

### hash vs history

**`hash` 模式**

1. URL 格式:

    - 使用 `#` 符号来表示路由，例如: `http://example.com/#/home` .
2. 浏览器行为：

    - `#` 后面的部分不会被发送到服务器，因此页面不会重新加载。
    - 通过监听 hashchange 事件来实现路由切换。

3. 兼容性：

    - 兼容性好，支持所有现代浏览器和一些老旧浏览器。
4. SEO：

    - 对于搜索引擎不友好，因为 # 后面的内容不会被搜索引擎索引。

**`history` 模式**

1. URL 格式：

    - 使用正常的 URL 路径，例如：http://example.com/home。

2. 浏览器行为：

    - 通过 HTML5 的 history.pushState 和 history.replaceState 方法来操作浏览器的历史记录。
    - 需要服务器配置支持，将所有路由请求重定向到入口 HTML 文件。
3. 兼容性：

    - 需要现代浏览器支持，不支持一些老旧浏览器。

4. SEO：

    - 对于搜索引擎友好，因为 URL 是标准的路径格式，搜索引擎可以索引这些路径。

uniapp

[/src/manifest.json](/src/manifest.json)
```json
{
  ...
  "h5": {
    "title": "小草壁纸",
    "router": {
      "mode": "history" // 或 "hash"
    }
  }
}
```


### uniCloud (history)

### GitHub Pages (uniapp/h5/history)
通过404.html实现单页应用的history模式

[/public/404.html](/public/404.html)
```html
```
  
[/vite.config.js](/vite.config.js)
```ts
import { defineConfig } from 'vite'
import uni from '@dcloudio/vite-plugin-uni'
import AutoImport from 'unplugin-auto-import/vite'

import { resolve } from 'path'
import { copyFileSync } from 'fs'

// https://vitejs.dev/config/
export default defineConfig({
  plugins: [
    uni(),
    AutoImport({
      imports: [
        'vue',
        'uni-app'
      ]
    }),
    {
      name: 'copy-404',
      closeBundle() {
        // 将 public/404.html 复制到输出目录
        const outputDir = resolve(__dirname, 'dist/build/h5');
        const sourceFile = resolve(__dirname, 'public/404.html');
        const destinationFile = resolve(outputDir, '404.html');
        copyFileSync(sourceFile, destinationFile);
        console.log('404.html copied to', destinationFile);
      }
    }
  ],
  build: {
    outDir: 'dist/build/h5', // 输出目录
    rollupOptions: {
      input: {
        main: 'index.html',
        // 其他入口文件
      }
    }
  }
})
```
1. 进入你的 GitHub 仓库页面。
2. 点击 "Settings"（设置）。
3. 在左侧菜单中找到 "Pages"（页面）。
4. 在 "Source" 部分，选择 "GitHub Actions" 作为构建和部署的来源。
5. 确保每个项目的 GitHub Pages URL 是唯一的，例如 `https://username.github.io/project-name/`

[/src/manifest.json](/src/manifest.json)
```json
{
  ...
  "h5": {
    "title": "小草壁纸",
    "router": {
      "mode": "history",
      "base": "/vue3_ln/" // 项目名
    }
  }
}
```
[/.github/workflows/deploy.yml](/.github/workflows/deploy.yml)
```yml
name: Deploy static content to Pages

on:
  push:
    branches: ["main"]
  workflow_dispatch:

permissions:
  contents: read
  pages: write
  id-token: write

concurrency:
  group: "pages"
  cancel-in-progress: false

jobs:
  upload:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout repository
      uses: actions/checkout@v2

    - name: Upload artifact
      uses: actions/upload-pages-artifact@v3
      with:
        path: './dist/build/h5/'
        # path: './dist/'

  deploy:
    needs: upload
    runs-on: ubuntu-latest
    environment:
      name: github-pages
      url: ${{ steps.deployment.outputs.page_url }}
    steps:
      - name: Checkout
        uses: actions/checkout@v4
      - name: Setup Pages
        uses: actions/configure-pages@v5
      - name: Download artifact
        uses: actions/download-artifact@v3
      - name: Deploy to GitHub Pages
        id: deployment
        uses: actions/deploy-pages@v4
        with:
          github_token: ${{ secrets.DEPLOY_TOKEN }}
          # github_token: ${{ secrets.GITHUB_TOKEN }}
          publish_dir: './dist/build/h5/'
          # publish_dir: './dist/'
```

### set history 重定向到入口 HTML 文件

#### if Nginx
```nginx
server {
    listen 80;
    server_name your_domain.com;

    location / {
        try_files $uri $uri/ /index.html;
    }

    location /static/ {
        # 静态文件的处理
        alias /path/to/static/files/;
    }
}
```
#### if Apache
.htaccess
```htaccess
<IfModule mod_rewrite.c>
    RewriteEngine On
    RewriteBase /
    RewriteRule ^index\.html$ - [L]
    RewriteCond %{REQUEST_FILENAME} !-f
    RewriteCond %{REQUEST_FILENAME} !-d
    RewriteRule . /index.html [L]
</IfModule>
```
#### if GitHub Pages
[404.html](/public/404.html)
```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>404</title>
  <script>
    window.location.href = '/index.html';
  </script>
</head>
<body>
  <h1>404 Not Found</h1>
</body>
</html>
```
